In [3]:
# This code is written by Hoplin
# Code last edited : 2020/06/20
# Description : This code will be used as algorithm of 'The Camp' News Sending program  

import os
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.request import URLError
from urllib.request import HTTPError
import requests
import sys
import time
import re
import json
import time
import asyncio

# Save News Datas : Data include articles, title
newsDatas = dict()
# News Topics will be save here
newsTopics = []

def returnHyperTextAddingHeader(URL):
    session = requests.Session()
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
    }
    html = session.get(URL,headers=headers).content 
    html = BeautifulSoup(html,"html.parser")
    return html

#각 섹션별 기사URL에 대해 기사 출력
def getArticleMainText(URL):
    '''
    html = urlopen(URL)
    html = BeautifulSoup(html,'html.parser')
    
    왜 이부분을 주석처리하고 사용하지 않나요?
    이 전에 다음과 같은 오류가 발생했었습니다.
    http.client.RemoteDisconnected: Remote end closed connection without response
    이러한 오류가 발생하는 이유는 요청을 받는 서버에서 요청을 비정상적이 요청으로 인지하고
    차단하는 경우에 발생하게 된다. 물론 서버 마다 다르지만 이렇게 차단된 상태의 html을 출력해 보면
    '비정상적인 요청에 따른 일시적 제한' 등의 글이 써져있는것을 볼 수 있다.
    결론적으로 말하면 뷰봇과 같은 '봇'으로 인식을 한 것이다.
    이렇게 봇으로 인식하는것을 방지하기 위해서는 패킷 header 정보에 User-Agent 정보를 넣어주면된다.
    '''

    html = returnHyperTextAddingHeader(URL)
    # 본문 내용은 id가 articleBodyContents 인 <div> 태그 안에있다.
    paragraphElements = re.sub(' +', ' ', html.find('div',{'id' : 'articleBodyContents'}).text).strip() # ' +' : white space가 한개 이상인 패턴을 찾는다.
    return paragraphElements
    
def JSONConverter(dataCapsule):
    with open('NaverNewsHeadlineScrape_' + str(time.strftime('%Y-%m-%d', time.localtime(time.time()))) + '.json','w',encoding="utf-8") as make_file:
        json.dump(dataCapsule,make_file,ensure_ascii=False,indent="\t") # indent : 들여쓰기값 Indent Parameter

try:
    naverNewsURL = 'https://news.naver.com/'
    #"https://news.naver.com/main/list.nhn?mode=LS2D&sid2={0}&sid1=101&mid=shm&date={1}&{2}".format("259", "20200408", "1")
    html = returnHyperTextAddingHeader(naverNewsURL)
    '''
    Get elements : About each news section. There are 7 sections. 
    
    1. Today headline news 
    
    2. Economy
    
    3. Society
    
    4. Life
    
    5. World
    
    6. IT
    
    7. Politics
    '''
    sectionElements = html.find('div',{'class' : 'main_content_inner _content_inner'}).findAll('div',{'class' : re.compile('main\_component[A-Za-z]*')})
    #print(sectionElements)
    del sectionElements[-1] # 맨 마지막에 필요 없는 데이터를 제거한다.
    
    # Last Scraped Time.
    newsDatas['lastScrapedTime'] = time.strftime('%c', time.localtime(time.time())) 
    # Source
    newsDatas['DataSource'] = 'Naver News'
    
    for topic in sectionElements:
        capsule = dict()
        topicName = topic.find('h4').text
        
        articlesInformation = topic.findAll('li')
        
        for inf in range(len(articlesInformation)):
            articleURL = articlesInformation[inf].find('a')['href']
            articleURLPatternChecker = re.compile('\/main\/[A-Za-z0-9]*')
            checkerBoolean = articleURLPatternChecker.match(articleURL)
            
            # Article Text Scrape Here
            
            # Headline News's URL Type : Full URL 
            # Other sections' News's URL Type : /main/~~~ 
            articText = None
            if checkerBoolean:
                articText = getArticleMainText('https://news.naver.com' + articleURL)
            else:
                articText = getArticleMainText(articleURL)
            capsule['article' + '_' +str(inf + 1)] = {
                'articleTitle' : articlesInformation[inf].find('a').text.strip().replace("\"","'"),
                'url' : articleURL,
                'articleText' : articText.replace("\t","").replace("\n","").replace("\"","'")
            }
                
        newsDatas[topicName] = capsule
    
    JSONConverter(newsDatas)
except URLError as errormessageURL:
    print(errormessageURL)
except HTTPError as errormessageHTTP:
    print(errormessageHTTP)


In [12]:
# This code is written by Hoplin
# Code last edited : 2020/06/20
# Description : This code will be used as algorithm of 'The Camp' News Sending program  

import os
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.request import URLError
from urllib.request import HTTPError
import requests
import sys
import time
import re
import json
import time
import asyncio
import ssl

# Save News Datas : Data include articles, title
newsDatas = dict()
# News Topics will be save here
newsTopics = []

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
}

#headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}

def returnHyperTextAddingHeader(URL):
    session = requests.Session()

    html = session.get(URL,headers=headers).content 
    html = BeautifulSoup(html,"html.parser")
    return html

#각 섹션별 기사URL에 대해 기사 출력
def getArticleMainText(URL):
    '''
    html = urlopen(URL)
    html = BeautifulSoup(html,'html.parser')
    
    왜 이부분을 주석처리하고 사용하지 않나요?
    이 전에 다음과 같은 오류가 발생했었습니다.
    http.client.RemoteDisconnected: Remote end closed connection without response
    이러한 오류가 발생하는 이유는 요청을 받는 서버에서 요청을 비정상적이 요청으로 인지하고
    차단하는 경우에 발생하게 된다. 물론 서버 마다 다르지만 이렇게 차단된 상태의 html을 출력해 보면
    '비정상적인 요청에 따른 일시적 제한' 등의 글이 써져있는것을 볼 수 있다.
    결론적으로 말하면 뷰봇과 같은 '봇'으로 인식을 한 것이다.
    이렇게 봇으로 인식하는것을 방지하기 위해서는 패킷 header 정보에 User-Agent 정보를 넣어주면된다.
    '''

    html = returnHyperTextAddingHeader(URL)
    # 본문 내용은 id가 articleBodyContents 인 <div> 태그 안에있다.
    paragraphElements = re.sub(' +', ' ', html.find('div',{'id' : 'articleBodyContents'}).text).strip() # ' +' : white space가 한개 이상인 패턴을 찾는다.
    return paragraphElements    

def JSONConverter(dataCapsule):
    with open('NaverNewsDailyScrape_' + str(time.strftime('%Y-%m-%d', time.localtime(time.time()))) + '.json','w',encoding="utf-8") as make_file:
        json.dump(dataCapsule,make_file,ensure_ascii=False,indent="\t") # indent : 들여쓰기값 Indent Parameter

try:
    idx = 0
    # 네이버 뉴스 - 경제(101) - 금융(259)
    target_url = "https://news.naver.com/main/list.nhn?mode=LS2D&sid2={0}&sid1=101&mid=shm&date={1}&{2}".format("259", "20200408", "1") 
    context = ssl._create_unverified_context()
    source_code = requests.get(target_url, headers = headers)

    soup = BeautifulSoup(source_code.text, "html.parser")
    url_set = set()
    for item in soup.find(id='main_content').find_all("a"):
        if item.has_attr("href"):
            if str(item.get("href")).startswith("http"):
    #             print(str(item.get("href")).find("sid1"))
    #             print(str(item.get("href"))[str(item.get("href")).find("sid1"):])
                url_set.add(str(item.get("href")))
 
    for url in url_set:
        
        #html은 내리지 않고 바로 json만 내리고 싶다면 아래 82-89블럭은 주석처리
        #html파일내리기 start
        print(url)
        source_code = requests.get(url, headers = headers)
        html_text = str(BeautifulSoup(source_code.text, "html.parser"))
        html_file = open("data/" + url[url.find("sid1"):] + '.html', 'w',-1,'utf-8')
        html_file.write(html_text)
        html_file.close()
        #end of html파일 내리기

        #json파일 만들기
        naverNewsURL = url
        html = returnHyperTextAddingHeader(naverNewsURL)

        idx+=1
        
        # Last Scraped Time.
        newsDatas['lastScrapedTime'] = time.strftime('%c', time.localtime(time.time())) 
        # Source
        newsDatas['DataSource'] = 'Naver News'

        capsule = dict()
        
        topicName = '네이버뉴스-경제-금융-20200408-'
        articleURL = url
        articleTitle = html.find('h3').text
        articText = getArticleMainText(url)

        capsule['article' + '_' +str(idx)] = {
            'articleTitle' : articleTitle.replace("\"","'"),
            'url' : articleURL,
            'articleText' : articText.replace("\t","").replace("\n","").replace("\"","'")
        }
        
        newsDatas[topicName+str(idx)] = capsule
    
    JSONConverter(newsDatas)
 
except URLError as errormessageURL:
    print(errormessageURL)
except HTTPError as errormessageHTTP:
    print(errormessageHTTP)




https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=259&oid=421&aid=0004573555
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=259&oid=468&aid=0000645976
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=259&oid=366&aid=0000506782
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=259&oid=119&aid=0002394759
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=259&oid=018&aid=0004616099
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=259&oid=277&aid=0004658238
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=259&oid=014&aid=0004405226
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=259&oid=014&aid=0004405223
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=259&oid=421&aid=0004573521
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=259&oid=119&aid=0002394758
https://news.naver.com/main/read.nhn?mod